In [1]:
from fuzzywuzzy import process
import re
from collections import deque
import itertools
from fuzzywuzzy import fuzz

def registerStreets(streets):
    streets["11th Street"].append("Eleventh Street")
    streets["2nd Avenue/Street"].append("Second Avenue/Street")
    streets["12th Street"].append("Twelfth Street")
    streets["8th Street"].append("Eight Street")
    streets["6th Street"].append("Sixth Street")
    streets["5th Street"].append("fifth Street")
    streets["3rd Avenue/Street"].append("Third Avenue/Street")
    streets["4th Avenue/Street"].append("Fourth Avenue/Street")
    streets["1st Street"].append("First Street")
    streets["10th Street"].append("Tenth Street")
    
with  open('./data/Detroitstreet.txt', 'r') as f:
    allstreets =[t[:-1].strip() for t in f.readlines()]
    streets = {x: [x] for x in allstreets}
    registerStreets(streets)
    for key in list(streets.keys()):
        stripname = key.replace(' Avenue/Road','').replace(' Avenue/Street','').replace(' Street','').replace(' Avenue','').replace(' Road','').replace(' Drive','')
        if (not stripname == key) and len(stripname)>2:
            streets[key].append(stripname)
    
def getscore(d, n, s):
    maxscore = 0
    for syn in d[s]:
        newscore =fuzz.token_set_ratio(n, syn)
        if newscore> maxscore:
            maxscore = newscore
    return maxscore

regex = re.compile('^([0-9]+)[ -](.+)$')

def ChangeIfneeded(n):
    global regex
    m = regex.match(n)
    return m.group(2)  if m is not None else n
    
    
def CheckNameold(n1, n2, s1, s2, d, rate=90):
    n1 = ChangeIfneeded(n1).replace(' BLVD',(' Boulevard'))
    n2 = ChangeIfneeded(n2).replace(' BLVD',(' Boulevard'))
    rate1 = getscore(d, n1, s1)
    rate2 = getscore(d, n2, s2)
    cruz1 = getscore(d, n2, s1)
    cruz2 = getscore(d, n1, s2)
    gl1 = rate1 + cruz1
    gl2 = rate2 + cruz2
    cand, score = (s1,gl1) if gl1>= gl2 else (s2, gl2)
    return cand if (score>=rate*2) else None

def CheckName(n1, n2, s1, s2, d, rate=90):
    return CheckNames((n1, n2), (s1, s2), d, rate)


def CheckNames(rawnames, tokens, d, rate=90):
    filterrawname =[]
    for rawname in rawnames:
        filterrawname.append(ChangeIfneeded(rawname).replace(' BLVD',(' Boulevard')))
        
    maxrate=0
    elementnumber = len(rawnames)
    limiterate = rate * elementnumber
    currtoken= None
    for token in tokens:
        currate =0
        for rawname in filterrawname:
            currate += getscore(d, rawname, token)
        if currate>=maxrate:
            maxrate = currate
            currtoken = token
    print(maxrate/elementnumber, currtoken)
    return currtoken if maxrate>= limiterate else None




print(CheckName('MILE','W. 7 MILE', '3 Mile Drive', '7 Mile Road', streets))
print(CheckName('32ND','32 MT', '32nd Street', 'Mt. Olivet', streets))
print(CheckName('THIRD','THIRD / HAZELWOOD', 'Hazelwood Avenue', '3rd Avenue/Street', streets))
print(CheckName('MT ELLIOTT/ 7 MILE','7 MILE AND MT. ELLIOTT', 'Mt. Elliott Avenue', '7 Mile Road', streets))
print(CheckName('W 7 MILE RD','W 7 MILE / KENTFIELD', '7 Mile Road', 'Field Avenue', streets))
print(CheckName('ANNABELLE','S. ANNABELLE # 1214', 'Annabelle Street', 'Ann Street', streets))
print(CheckName('LILLIBRIDGE','LILLIBRIDGE', 'Ridge Street', 'Lillbridge Avenue', streets))
print(CheckName("18911 Rolandale","18900 ROLENDALE","Rolandale","Dale Avenue", streets))
print(CheckName("00 AUBUDON AND WAVENEY","Waveney","Waveney Avenue","Audubon Avenue", streets))
print(CheckName("7485 West Parkway","7485 W Parkway St","West Parkway","Park Street", streets))
print(CheckName("15114 EASTWOOD","00 HAYES EASTWWOD","Hayes Avenue","Eastwood Avenue", streets))
print(CheckName("6131 ASHTON","00 ASHTON / DAYTON","Ashton Avenue/Road","Ash Street", streets))
print(CheckName("00 W.GRAND BLVD/GRANDRIVER","00 GRAND RIVER / W GRAND BLVD","Grand Boulevard","Grand River", streets))
print(CheckName("3521 ROSA PARKS BLVD","03500 ROSA PARKS APT 202","Rosa Parks Boulevard","Park Street", streets))

100.0 7 Mile Road
7 Mile Road
57.5 32nd Street
None
75.0 3rd Avenue/Street
None
100.0 Mt. Elliott Avenue
Mt. Elliott Avenue
100.0 7 Mile Road
7 Mile Road
100.0 Annabelle Street
Annabelle Street
95.0 Lillbridge Avenue
Lillbridge Avenue
94.5 Rolandale
Rolandale
100.0 Waveney Avenue
Waveney Avenue
91.5 West Parkway
West Parkway
82.0 Eastwood Avenue
None
100.0 Ashton Avenue/Road
Ashton Avenue/Road
100.0 Grand Boulevard
Grand Boulevard
85.5 Rosa Parks Boulevard
None


In [42]:
import sqlite3
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()
#query="select f.Id, f.Address, f.StreetName as street1, s.Id, s.Address, s.StreetName as street2 from LocationReferenced as f ,LocationReferenced as s where f.ClosestLocation = s.Id and street1!=street2 order by street1;"
query="select id1, id2, ad1, ad2, name1, name2, RefHash from (select RefHash, min (Id) as id1, max(Id) as id2,  max (Address) as ad1, min(Address) as ad2, max(StreetName)  as name1,min(StreetName) as name2,  count  ( distinct StreetName)  as streetCount from LocationReferenced where IsMaster=1 group by RefHash) where name1!=name2  and streetCount=2 ;"
queryupdate = "update LocationReferenced set StreetName='{}' where RefHash='{}';"

first = True
rows = cur.execute(query).fetchall()[:]
for row in rows:
    fid, sid, sadress, fadress, fsname, ssname, refhash = row
    if first:
        first = False
        print(fid, fadress, fsname)
        print(sid, sadress, ssname)
        print(len(rows))
    newname = CheckName(fadress, sadress, fsname, ssname, streets, 78.5)
    if newname is not None:        
        #sql = queryupdate.format(newname,refhash)
        #nc = cur.execute(sql)
        print('Changed', newname, fadress,', ',sadress,fid, sid) 
        con.commit()
con.close()

380541 00 LINCOLN PARK PD Lincoln Avenue
422883 01400 CLEOPHUS C Street
723
Changed South Street 08400 SOUTH ,  8421 ST 8789 443809
Changed South Street 08200 SOUTH STREET ,  8268 ST 8790 331737
Changed South Street 08000 SOUTH ST. ,  8105 ST 8792 371456
Changed Hammond Street 04600 HAMMOND ,  04600 HANNAND 361721 427645
Changed Michigan Avenue 5622 MICHIGAN ,  5671 Chopin 70654 326494
Changed 2nd Avenue/Street 00 SECOND/W WILLIS ,  4202 SECOND 31826 440299
Changed Beaubien Street 1501-1699 Beaubien Street 48226, ,  454-498 Madison Street 48226, 314844 315933
Changed Greenway Avenue 1820 Mack Avenue 48207, ,  Dequindre Cut Greenway Trail 48207, 310582 314755
Changed Meyers Road 00 MEYER / SCHOOLCRAFT ,  13654 MEYERS 11589 430294
Changed Marlowe Street 00 MARLOW/ CLARITA ,  18716 MARLOWE 57946 412396
Changed Sussex Street 00 CAMBRIDGE / SUSEX ,  19300 SUSSEX 353936 358822
Changed Glendale Avenue 00 GELNDALE AND LASALLE ,  12800 GLENDALE 334870 426000
Changed Whittier Road 00 WHITTER AND

In [38]:
con.close()


In [4]:
import sqlite3
con = sqlite3.connect('./data/location.s3db')
cur = con.cursor()
query="select RefHash from (select RefHash from ValidLocationReferenced group by RefHash having count(distinct StreetName) > 1) "
queryids = "select StreetName, Address from ValidLocationReferenced where Refhash = '{}'"
queryupdate = "update LocationReferenced set StreetName=? where RefHash=?;"

first = True
rows = cur.execute(query).fetchall()[:]
for row in rows:
    refhash = row[0]
    names =[]
    tokens= set()
    for innerrow in cur.execute(queryids.format(refhash)):
        token,  name = innerrow
        names.append(name)
        tokens.add(token)
    newtoken = CheckNames(names, tokens, streets)
    if newtoken is not None:        
        nc = cur.execute(queryupdate,(newtoken,refhash))
        print('Changed:', names, tokens, newtoken) 
        con.commit()
con.close()

KeyboardInterrupt: 

In [17]:
import sqlite3

def check(cur, con, refhash, rate=90, add=None):
    queryids = "select StreetName, Address from ValidLocationReferenced where Refhash = '{}';"
    queryupdate = "update LocationReferenced set StreetName=? where RefHash=?;"
    names =[]
    tokens= set()
    if add is not None:
        tokens.add(add)
    for innerrow in cur.execute(queryids.format(refhash)):
        token,  name = innerrow
        names.append(name)
        tokens.add(token)
    print(tokens, names)
    newtoken = CheckNames(names, tokens, streets, rate)
    if newtoken is not None:        
        nc = cur.execute(queryupdate,(newtoken,refhash))
        print('Changed:', names, tokens, newtoken) 
        con.commit()
        
def singlecheck(refhash, rate=90, add=None):
    con = sqlite3.connect('./data/location.s3db')
    cur = con.cursor()
    return check(cur, con, refhash, rate, add)
    

In [94]:
singlecheck('dpsbgrnv7uppuntmtdy7hc0002000000',85)

None
{'Joy Road', 'Field Avenue', 'Meyers Road'} ['12701 JOY ROAD', '12701 JOY ROAD', '12701 JOY ROAD', '12701 JOY ROAD', '12700 JOY ROAD', '12701 JOY ROAD', '00 MEYERS/JOY RD', '00 SOUTHFIELD POLICE DEPT', '00 SOUTHFIELD POLICE DEPT', '00 MEYERS AND JOY RD']
85.4 Joy Road
Changed: ['12701 JOY ROAD', '12701 JOY ROAD', '12701 JOY ROAD', '12701 JOY ROAD', '12700 JOY ROAD', '12701 JOY ROAD', '00 MEYERS/JOY RD', '00 SOUTHFIELD POLICE DEPT', '00 SOUTHFIELD POLICE DEPT', '00 MEYERS AND JOY RD'] {'Joy Road', 'Field Avenue', 'Meyers Road'} Joy Road
